In [1]:
import numpy as np 
import pandas as pd
import urllib
import json
import codecs
import pandas as pd
import json
import time
from tqdm import tqdm


def swe2eng(swe_str):
    swe_characetrs = ['å','ä', 'ö', 'Å', 'Ä', 'Ö'] 
    eng_characters = ['a', 'a', 'o', 'A', 'A', 'O']
    
    eng_str = swe_str 
    
    for i in range(len(swe_characetrs)):
        eng_str = eng_str.replace(swe_characetrs[i], eng_characters[i])

    return eng_str

# Google API Key, You must have at least one API key associated with your project. 
f = open('apikey.txt', 'r')
key = f.readline()
f.close()


In [2]:
country = 'swe'

doc2 = codecs.open('population_municipality.txt','rU') #open for reading with "universal" type set

population_municipality = pd.read_csv(doc2, sep='\t')
population_municipality['postal_code'] = [population_municipality['region'][i][0:population_municipality['region'][i].find(' ')] 
                                          for i in range(population_municipality.shape[0])]
population_municipality['city'] = [population_municipality['region'][i][population_municipality['region'][i].find(' ')+1:] 
                                   for i in range(population_municipality.shape[0])]

population_municipality = population_municipality.rename(columns = {'Population':'population', 'city':'municipality'})
population_municipality['country']=country
population_municipality['latitude'] = np.zeros(len(population_municipality['municipality']))
population_municipality['longitude'] = np.zeros(len(population_municipality['municipality']))
population_municipality = population_municipality[['country','postal_code', 'municipality', 'latitude',
                                                   'longitude', 'population', 'year']]

In [3]:
municipalityList = range(len(population_municipality['municipality']))

for municipality in tqdm(municipalityList):
    address=swe2eng(population_municipality['municipality'][municipality].replace(' ', '+'))
    url='https://maps.googleapis.com/maps/api/geocode/json?address=' + address + 'Sweden&key=' + key
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    if data['status']=='OK' and data['results'][0]['address_components'][-1]['long_name'] == 'Sweden':
        population_municipality['latitude'][municipality] = data['results'][0]['geometry']['location']['lat']
        population_municipality['longitude'][municipality] = data['results'][0]['geometry']['location']['lng']

population_municipality.to_csv('population_municipality.csv', index=False) 

  0%|                                                                                          | 0/290 [00:00<?, ?it/s]C:\Users\hshokri\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\hshokri\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
100%|████████████████████████████████████████████████████████████████████████████████| 290/290 [01:12<00:00,  2.18it/s]


In [10]:
population_municipality

,country,postal_code,municipality,latitude,longitude,population,year
0,swe,0114,Upplands Väsby,59.519610,17.928340,46786,2019
1,swe,0115,Vallentuna,59.535700,18.078017,34090,2019
2,swe,0117,Österåker,59.480495,18.290296,45574,2019
3,swe,0120,Värmdö,59.316366,18.446572,45000,2019
4,swe,0123,Järfälla,59.425265,17.823897,79990,2019
5,swe,0125,Ekerö,59.359052,17.669494,28690,2019
6,swe,0126,Huddinge,59.209773,18.052167,112848,2019
7,swe,0127,Botkyrka,59.144838,17.840658,94606,2019
8,swe,0128,Salem,59.207680,17.774222,16750,2019
9,swe,0136,Haninge,0.000000,0.000000,92095,2019
